In [76]:
pip install xgboost


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [77]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [78]:
pip install -U imbalanced-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [79]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt

In [80]:
SEED = 27 #611240 for lgbm
N_FOLDS = 5
TARGET = 'Target'

List Model
- Balanced Random Forest
- Random Forest
- Light GBM
- Catboost


In [81]:
train=pd.read_csv('D:/Local Disk C/Python project/src/Tugas/EAS PSD/train.csv')
test=pd.read_csv('D:/Local Disk C/Python project/src/Tugas/EAS PSD/test.csv')

In [82]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              76518 non-null  int64  
 1   Marital status                                  76518 non-null  int64  
 2   Application mode                                76518 non-null  int64  
 3   Application order                               76518 non-null  int64  
 4   Course                                          76518 non-null  int64  
 5   Daytime/evening attendance                      76518 non-null  int64  
 6   Previous qualification                          76518 non-null  int64  
 7   Previous qualification (grade)                  76518 non-null  float64
 8   Nacionality                                     76518 non-null  int64  
 9   Mother's qualification                 

In [83]:
num_cols = train._get_numeric_data().columns
num_cols

Index(['id', 'Marital status', 'Application mode', 'Application order',
       'Course', 'Daytime/evening attendance', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units

In [84]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51012 entries, 0 to 51011
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              51012 non-null  int64  
 1   Marital status                                  51012 non-null  int64  
 2   Application mode                                51012 non-null  int64  
 3   Application order                               51012 non-null  int64  
 4   Course                                          51012 non-null  int64  
 5   Daytime/evening attendance                      51012 non-null  int64  
 6   Previous qualification                          51012 non-null  int64  
 7   Previous qualification (grade)                  51012 non-null  float64
 8   Nacionality                                     51012 non-null  int64  
 9   Mother's qualification                 

In [85]:
cat_features = [
    'Marital status',
    'Application mode',
    'Course',
    'Previous qualification',
    'Nacionality',
    "Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation"
]



In [86]:
train['Target'].unique()

array(['Graduate', 'Dropout', 'Enrolled'], dtype=object)

In [87]:
for i in cat_features:
  print(i)
  print(train[i].nunique())

Marital status
6
Application mode
22
Course
19
Previous qualification
21
Nacionality
18
Mother's qualification
35
Father's qualification
39
Mother's occupation
40
Father's occupation
56


Split which feature that neede to one hot or label encoding

In [88]:
label_features = ["Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation",
    'Marital status']
onehot_features = ['Application mode',
    'Course',
    'Previous qualification',
    'Nacionality']

In [89]:
# # One-hot encoding
# for cat in onehot_features:
#     onehots = pd.get_dummies(test[cat], prefix=cat)
#     test = test.join(onehots)

# for cat in onehot_features:
#     onehots_tr = pd.get_dummies(train[cat], prefix=cat)
#     train = train.join(onehots_tr)

# # Drop original columns
# encode_data_test = test.drop(columns=onehot_features).copy()

# encode_data_train = train.drop(columns=onehot_features).copy()
# # Fill NaN values with 0
# encode_data_test.fillna(0, inplace=True)
# bool_cols = encode_data_test.select_dtypes(include='bool').columns.tolist()
# encode_data_test[bool_cols] = encode_data_test[bool_cols].astype(float)

# encode_data_train.fillna(0, inplace=True)
# bool_cols = encode_data_train.select_dtypes(include='bool').columns.tolist()
# encode_data_train[bool_cols] = encode_data_train[bool_cols].astype(float)

In [90]:
# for col in cat_features:
#     train[col] = train[col].astype('category')
#     test[col] = test[col].astype('category')

In [91]:
# one_hot = OneHotEncoder()

In [92]:
# encode_data_train.head()

In [93]:
# X = encode_data_train.drop(TARGET, axis=1)
# y = encode_data_train[TARGET]

In [94]:
X = train.drop(TARGET, axis=1)
y = train[TARGET]

In [95]:
y.head()

0    Graduate
1     Dropout
2     Dropout
3    Enrolled
4    Graduate
Name: Target, dtype: object

In [96]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [97]:
y_resampled.head()

0    Graduate
1     Dropout
2     Dropout
3    Enrolled
4    Graduate
Name: Target, dtype: object

In [98]:
label_encoder = LabelEncoder()
y_resampled = label_encoder.fit_transform(y_resampled)

In [99]:
scaler = StandardScaler()
scaler.fit(X_resampled)
X_train_scaled = pd.DataFrame(scaler.transform(X_resampled), columns=X_resampled.columns)

In [124]:
one_hot = OneHotEncoder()
one_hot.fit(X_train_scaled[onehot_features])
one_hot_encoded = one_hot.transform(X_train_scaled[onehot_features]).toarray()

In [101]:
# # Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_resampled, test_size=0.2, random_state=42)



In [102]:
SEED = 27
xgb_params = {
    'n_jobs': -1,
    'max_bin': 86,
    'verbosity': 0,
    'max_depth': 5,
    'max_leaves': 64,
    'booster': 'gbtree',
    'n_estimators': 1539,
    'random_state': SEED,
    'enable_categorical': True,
    'gamma': 0.9069384135972282,
    'reg_alpha': 9.076826031603096,
    'subsample': 0.9335856723937552,
    'reg_lambda': 3.7191495119106976,
    'max_delta_step': 1.1802158903460869,
    'min_child_weight': 8.745168792677534,
    'colsample_bynode': 0.8390631221126779,
    'colsample_bytree': 0.5534666374509267,
    'colsample_bylevel': 0.5695267422612745,
}

cb_params = {
    'depth': 9,
    'verbose': False,
    'thread_count': -1,
    'iterations': 1830,
    'border_count': 200,
    'random_state': SEED,
    'min_child_samples': 4,
    'bootstrap_type': 'Bernoulli',  # Supported type
    'grow_policy': 'Depthwise',
    'l2_leaf_reg': 4.349614487163372,
    'random_strength': 0.16489500172653238,
    'cat_features': cat_features,
    'task_type': 'GPU',  # Ensure GPU usage
    'devices': '0'  # Specify GPU device
}

lgbm_params = {
    'n_jobs': -1,
    'verbose': -1,
    'max_depth': 77,
    'num_leaves': 42,
    'subsample_freq': 8,
    'n_estimators': 169,
    'random_state': SEED,
    'min_child_samples': 199,
    'device': 'gpu',
    'subsample': 0.9178993412905838,
    'reg_alpha': 0.9200214666236493,
    'reg_lambda': 0.2652568189927319,
    'min_child_weight': 0.4410010971781976,
    'colsample_bytree': 0.35379935570036725,

}

hgb_params = {
    'verbose': 0,
    'max_depth': 4,
    'max_bins': 250,
    'max_iter': 1522,
    'max_leaf_nodes': 92,
    'random_state': SEED,
    'min_samples_leaf': 44,
    'l2_regularization': 0.4495943539321024,
}

gb_params = {
    'verbose': 0,
    'max_depth': 4,
    'subsample': 0.9,
    'n_estimators': 600,
    'random_state': SEED,
    'learning_rate': 0.05,
}

rf_params = {
    'verbose': 0,
    'max_depth': None,
    'n_estimators': 1200,
    'random_state': SEED,
}

In [103]:
from sklearn.ensemble import VotingClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
# Initialize individual models with GPU for XGBoost, LightGBM, and CatBoost
xgboost = XGBClassifier(**xgb_params)
lightgbm = LGBMClassifier(**lgbm_params)
# catboost = CatBoostClassifier(**cb_params)
hist_gradient_boosting = HistGradientBoostingClassifier(**hgb_params)
gradient_boosting = GradientBoostingClassifier(**gb_params)
random_forest = RandomForestClassifier(**rf_params)

# Create a VotingClassifier with the models
voting_clf = VotingClassifier(
    estimators=[
        ('xgb', xgboost),
        # ('cat', catboost),
        ('lgb', lightgbm),
        ('hgb', hist_gradient_boosting),
        ('gb', gradient_boosting),
        ('rf', random_forest)
    ],
    voting='soft'
)

# Train the VotingClassifier
voting_clf.fit(one_hot_encoded, y_resampled)

# Predict and evaluate
# y_pred = voting_clf.predict(X_val)
# accuracy = accuracy_score(y_val, y_pred)
# print(f"Voting Classifier Accuracy: {accuracy:.4f}")
# print(classification_report(y_val, y_pred))

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster='gbtree',
                                            callbacks=None,
                                            colsample_bylevel=0.5695267422612745,
                                            colsample_bynode=0.8390631221126779,
                                            colsample_bytree=0.5534666374509267,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=True,
                                            eval_metric=None,
                                            feature_types=None,
                                            gamma=0.9069384135972282,
                                            grow_policy=None,
                                            im...
                              HistGradientBoostingClassifier(l2_regularization=0.4495943539321024,
                                                             max_bins=250,
                                                             max_depth=4,
                                                             max_iter=1522,
                                                             max_leaf_nodes=92,
                                                             min_samples_leaf=44,
                                                             random_state=27)),
                             ('gb',
                              GradientBoostingClassifier(learning_rate=0.05,
                                                         max_depth=4,
                                                         n_estimators=600,
                                                         random_state=27,
                                                         subsample=0.9)),
                             ('rf',
                              RandomForestClassifier(n_estimators=1200,
                                                     random_state=27))],
                 voting='soft')

In [121]:
from joblib import Parallel, delayed 
import joblib 


# Save the model as a pickle in a file 
joblib.dump(voting_clf, 'filename.pkl') 

# Load the model from the file 
voting_from_joblib = joblib.load('filename.pkl') 

# Use the loaded model to make predictions 
# voting_from_joblib.predict(X_test) 


In [118]:
test = pd.read_csv('D:/Local Disk C/Python project/src/Tugas/EAS PSD/test.csv')
scaler = StandardScaler()
scaler.fit(test)
test = pd.DataFrame(scaler.transform(test), columns=test.columns)
one_hot = OneHotEncoder()
one_hot.fit(test[onehot_features])
one_hot_encoded_test = one_hot.transform(test[onehot_features]).toarray()

In [ ]:
# one_hot_df = pd.DataFrame(one_hot_encoded, columns=one_hot.get_feature_names(onehot_features))

In [119]:
import pandas as pd

# Ensure the test data has the same preprocessing as the training data
# for col in cat_features:
#     test[col] = test[col].astype('category')

# Make predictions on the test data
test_predictions = voting_clf.predict(one_hot_encoded_test)

# Load the sample submission file to ensure the correct format
sample_submission = pd.read_csv('D:/Local Disk C/Python project/src/Tugas/EAS PSD/sample_submission.csv')
# Create the submission DataFrame
submission = pd.DataFrame({
    'id': sample_submission['id'],  # Ensure the 'id' column matches the sample submission
    'Target': label_encoder.inverse_transform(test_predictions)  # Inverse transform if necessary
    
})
# submission['Target'] = submission['Target'].replace('Graduate',1).replace('Dropout',0).replace('Enrolled',2)
# Save the submission file
submission.to_csv('submission1.csv', index=False)

print("Submission file created: submission.csv")


ValueError: Feature shape mismatch, expected: 967, got 79